### Installation
Install the packages required for executing this notebook.

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-aiplatform {USER_FLAG} -q
! pip3 install -U google-cloud-storage {USER_FLAG} -q
! pip3 install {USER_FLAG} kfp google-cloud-pipeline-components --upgrade -q

## Restart the kernel
Once you've installed the additional packages, you need to restart the notebook kernel so it can find the packages.

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

Check the versions of the packages you installed. The KFP SDK version should be >=1.6.

In [ ]:
! python3 -c "import kfp; print('KFP SDK version: {}'.format(kfp.__version__))"

In [ ]:
import kfp
from kfp.v2 import dsl
from kfp.v2.dsl import component
from kfp.v2.dsl import (Artifact,
                        Dataset,
                        Input,
                        Model,
                        Output,
                        Metrics,
                        ClassificationMetrics,
                        component, 
                        OutputPath, 
                        InputPath)
import google.cloud.aiplatform as aip
from google_cloud_pipeline_components import aiplatform as gcc_aip
from kfp.v2.components import importer_node
from google_cloud_pipeline_components.types import artifact_types

#### Project and Pipeline Configurations

In [ ]:
#The Google Cloud project that this pipeline runs in.
project_id = "de-2022-366209"
# The region that this pipeline runs in
region = "us-west1"
# Specify a Cloud Storage URI that your pipelines service account can access. The artifacts of your pipeline runs are stored within the pipeline root.
pipeline_root_path = "gs://de-group9-bucket"

#### Create Pipeline Components

We can create a component from Python functions (inline) and from a container. We will first try inline python functions. 

Step 1: Define the python function

Step 2:  Use **kfp.components.create_component_from_func** build the component. This function takes four parameters.

**1.func**: The Python function to convert.

**2.base_image**: (Optional.) Specify the Docker container image to run this function in. 

**3.output_component_file**: (Optional.) Writes your component definition to a file. 

**4.packages_to_install**: (Optional.) A list of versioned Python packages to install before running your function.

Another thing we need to consider is passing parameters between components. We can pass simple parameters such as integer, string, tuple, dict, and list by values. To pass the large datasets or complex configurations, we can use files. We can annotate the Python function’s parameters to indicate input or output files for the component. 

Refer to  https://www.kubeflow.org/docs/components/pipelines/sdk/python-function-components/ for more information.

#### Pipeline Component : Data Ingestion

In [ ]:
@component(
    packages_to_install=["pandas","google-cloud-storage"],
    base_image="python:3.10.7-slim",
    output_component_file="download_data.yaml"
)
def download_data(project_id: str, bucket: str, file_name: str, dataset: Output[Dataset]):
    '''download data'''
    from google.cloud import storage
    import pandas as pd
    import logging 
    import sys
    
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    
    # Downloaing the file from a google bucket 
    client = storage.Client(project=project_id)
    bucket = client.get_bucket(bucket)
    blob = bucket.blob(file_name)
    blob.download_to_filename(dataset.path + ".csv")
    logging.info('Downloaded Data!')

#### Pipeline Component : Training-KNN 

In [ ]:
@component(
    packages_to_install=['google-cloud-storage', 'pandas', 'keras', 'tensorflow', 'h5py', 'scikit-learn'],
    base_image="python:3.10.7-slim",
    output_component_file="training_mlp.yaml"
)
def train_knn (features: Input[Dataset], project_id: str, model_repo: str)-> dict:
    '''train a KNN with default parameters'''
    import pandas as pd
    from sklearn.metrics import accuracy_score
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn import metrics
    from sklearn.model_selection import train_test_split
    from google.cloud import storage
    from keras.layers import Dense
    from keras.models import Sequential
    import json
    import logging 
    import sys
    import os
    import pickle
    
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    
    df = pd.read_csv(features.path+".csv")
    
    logging.info(df.columns)
    
    train, test = train_test_split(df, test_size = 0.25)
    
    # split into input (X) and output (Y) variables
    train_X = train[['Sepal.Length', 'Sepal.Width', 'Petal.Length', 'Petal.Width']]
    train_y = train.Species
    test_X = test[['Sepal.Length', 'Sepal.Width', 'Petal.Length','Petal.Width']]
    test_y = test.Species
    
    # define model
    model = KNeighborsClassifier(n_neighbors=5)     
    #fit the model
    model.fit(train_X,train_y)
    y_pred = model.predict(test_X)
    
    #logging.info(model.metrics_names)
    # metrics = ("Accuracy Score:", accuracy_score(test_y,y_pred))
    metrics = {
        "accuracy": accuracy_score(test_y,y_pred)
    }
    logging.info(metrics)
    
    local_file = '/tmp/local_model.h5'
    pickle.dump(model, open(local_file, "wb"))     
    # write out output
  
    # Save to GCS as model.h5
    client = storage.Client(project=project_id)
    bucket = client.get_bucket(model_repo)
    blob = bucket.blob('knnmodel.h5')
    # Upload the locally saved model
    blob.upload_from_filename(local_file)

    print("Saved the model to GCP bucket : " + model_repo)
    return metrics

#### Pipeline Component : Training LogisticRegression

In [ ]:
@component(
    packages_to_install=['google-cloud-storage', 'pandas', 'scikit-learn'],
    base_image="python:3.10.7-slim",
    output_component_file="train_lr_model.yaml"
)
def train_lr (features: Input[Dataset], project_id: str, model_repo: str)-> dict:
    '''train a LogisticRegression with default parameters'''
    import pandas as pd
    from sklearn.linear_model import LogisticRegression
    from sklearn import metrics
    from sklearn.model_selection import train_test_split
    from google.cloud import storage
    import json
    import logging 
    import sys
    import os
    import pickle  
       
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    
    df = pd.read_csv(features.path+".csv")
    
    logging.info(df.columns)      

    train, test = train_test_split(df, test_size = 0.25)

    # split into input (X) and output (Y) variables
    train_X = train[['Sepal.Length', 'Sepal.Width', 'Petal.Length', 'Petal.Width']]
    train_y = train.Species
    test_X = test[['Sepal.Length', 'Sepal.Width', 'Petal.Length','Petal.Width']]
    test_y = test.Species

    model = LogisticRegression()
    model.fit(train_X, train_y)
    
    metrics = {
        "accuracy": model.score(test_X, test_y)
    }
    logging.info(metrics)
   
    # Save the model localy
     
    local_file = '/tmp/local_model.h5'
    pickle.dump(model, open(local_file, "wb"))     
    # write out output
  
    # Save to GCS as model.h5
    client = storage.Client(project=project_id)
    bucket = client.get_bucket(model_repo)
    blob = bucket.blob('lrmodel.h5')
    # Upload the locally saved model
    blob.upload_from_filename(local_file)

    print("Saved the model to GCP bucket : " + model_repo)
    return metrics

#### Pipeline Component : Prediction-KNN

In [ ]:
@component(
    packages_to_install=['google-cloud-storage', 'pandas', 'keras', 'tensorflow', 'h5py',  'scikit-learn'],
    base_image="python:3.10.7-slim",
    output_component_file="predict_knn.yaml"
)
def predict_knn(project_id: str, model_repo: str, features: Input[Dataset], results: Output[Dataset]):
    import pandas as pd
    from keras.models import load_model
    from google.cloud import storage
    from sklearn import metrics
    import json
    import logging
    import sys
    import os
    import pickle

    logging.basicConfig(stream=sys.stdout, level=logging.INFO)

    df = pd.read_csv(features.path+".csv")

    xNew = df.loc[:, ["Sepal.Length","Sepal.Width","Petal.Length","Petal.Width"]].values
    client = storage.Client(project=project_id)
    bucket = client.get_bucket(model_repo)
    blob = bucket.blob('knnmodel.h5')
    blob.download_to_filename('/tmp/local_model.h5')
    model = load_model('/tmp/local_model.h5')
    dfcp = df.copy()
    result = model.predict(xNew)
    y_classes = result.argmax(axis=-1)
    logging.info(y_classes)
    dfcp['pclass'] = y_classes.tolist()
    dfcp.to_csv(results.path + ".csv" , index=False, encoding='utf-8-sig')

In [ ]:
@component(
    packages_to_install=['google-cloud-storage', 'pandas', 'keras', 'tensorflow', 'h5py',  'scikit-learn'],
    base_image="python:3.10.7-slim",
    output_component_file="save_knn.yaml"
)
def save_knn(project_id: str, model_repo: str, features: Input[Dataset], results: Output[Dataset]):
    import pandas as pd
    from keras.models import load_model
    from google.cloud import storage
    from sklearn import metrics
    import json
    import logging
    import sys
    import os
    import pickle

    logging.basicConfig(stream=sys.stdout, level=logging.INFO)

    #open the model
    client = storage.Client(project=project_id)
    bucket = client.get_bucket(model_repo)
    blob = bucket.blob('knnmodel.h5')
    blob.download_to_filename('/tmp/local_model.h5')
    
    #Loading the saved model
    model = pickle.load(open('/tmp/local_model.h5', 'rb'))

    # Save the model localy
    local_file = '/tmp/local_model2.h5'
    pickle.dump(model, open(local_file, "wb"))

    # Save to GCS as model.h5
    client = storage.Client(project=project_id)
    bucket = client.get_bucket(model_repo)
    blob = bucket.blob('model.h5')

    # Upload the locally saved model
    blob.upload_from_filename(local_file)


    print("Saved the model to GCP bucket : " + model_repo)

#### Pipeline Component : Prediction-LR

In [ ]:
@component(
    packages_to_install=['google-cloud-storage', 'pandas', 'scikit-learn'],
    base_image="python:3.10.7-slim",
    output_component_file="predict_lr.yaml"
)
def predict_lr(project_id: str, model_repo: str, features: Input[Dataset], results: Output[Dataset]):
    import pandas as pd
    import pickle
    from google.cloud import storage
    import json
    import logging
    import sys
    import os
    import pickle

    logging.basicConfig(stream=sys.stdout, level=logging.INFO)

    df = pd.read_csv(features.path+".csv")

    client = storage.Client(project=project_id)
    bucket = client.get_bucket(model_repo)
    blob = bucket.blob('lrmodel.h5')
    filename = '/tmp/local_model.h5'
    blob.download_to_filename(filename)
    
    
    #Loading the saved model
    model = pickle.load(open(filename, 'rb'))
    
    xNew = df[["Sepal.Length","Sepal.Width","Petal.Length","Petal.Width"]]
    
    dfcp = df.copy()
    y_classes = model.predict(xNew)
    logging.info(y_classes)
    dfcp['pclass'] = y_classes.tolist()
    dfcp.to_csv(results.path + ".csv" , index=False, encoding='utf-8-sig')

In [ ]:
@component(
    packages_to_install=['google-cloud-storage', 'pandas', 'scikit-learn'],
    base_image="python:3.10.7-slim",
    output_component_file="save_lr.yaml"
)
def save_lr(project_id: str, model_repo: str, features: Input[Dataset], results: Output[Dataset]):
    import pandas as pd
    import pickle
    from google.cloud import storage
    import json
    import logging
    import sys
    import os
    
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    
    #open the model
    client = storage.Client(project=project_id)
    bucket = client.get_bucket(model_repo)
    blob = bucket.blob('lrmodel.h5')
    filename = '/tmp/local_model.h5'
    blob.download_to_filename(filename)
    
    #Loading the saved model
    model = pickle.load(open(filename, 'rb'))
    
    # Save the model localy
    local_file = '/tmp/local_model2.h5'
    with open(local_file, 'wb') as file:
        pickle.dump(model, file)
    
    # Save to GCS as model.h5
    client = storage.Client(project=project_id)
    bucket = client.get_bucket(model_repo)
    blob = bucket.blob('model.h5')
    
    # Upload the locally saved model
    blob.upload_from_filename(local_file)
    
    print("Saved the model to GCP bucket : " + model_repo)

#### Pipeline Component : Algorithm Selection 

In [ ]:
@component(
    base_image="python:3.10.7-slim",
    output_component_file="compare_model.yaml"
)
def compare_model(knn_metrics: dict, lr_metrics: dict) -> str:
    import logging
    import json
    import sys
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    logging.info(knn_metrics)
    logging.info(lr_metrics)
    if knn_metrics.get("accuracy") > lr_metrics.get("accuracy"):
        return "KNN"
    else :
        return "LR"

#### Define the Pipeline

In [ ]:
#Define the workflow of the pipeline.
@kfp.dsl.pipeline(
    name="iris-prdictor-training-pipeline-v2",
    pipeline_root=pipeline_root_path)
def pipeline(project_id: str, data_bucket: str, trainset_filename: str, model_repo: str, testset_filename: str, ):
    
    di_op = download_data(
        project_id=project_id,
        bucket=data_bucket,
        file_name=trainset_filename
    )
    
    training_knn_job_run_op = train_knn(
        project_id=project_id,
        model_repo=model_repo,       
        features=di_op.outputs["dataset"]
    )
    
    training_lr_job_run_op = train_lr(
        project_id=project_id,
        model_repo=model_repo,       
        features=di_op.outputs["dataset"]
    )
    
    pre_di_op = download_data(
        project_id=project_id,
        bucket=data_bucket,
        file_name=testset_filename
    ).after(training_knn_job_run_op, training_lr_job_run_op)
    
    comp_model__op = compare_model(training_knn_job_run_op.output,
                                       training_lr_job_run_op.output).after(training_knn_job_run_op, training_lr_job_run_op)  
    
    # defining the branching condition
    with dsl.Condition(comp_model__op.output=="KNN"):
        predict_knn_job_run_op = save_knn(
        project_id=project_id,
        model_repo=model_repo,       
        features=pre_di_op.outputs["dataset"]
        )
    with dsl.Condition(comp_model__op.output=="LR"):
        predict_lr_job_run_op = save_lr(
        project_id=project_id,
        model_repo=model_repo,       
        features=pre_di_op.outputs["dataset"]
       )

#### Compile the pipeline into a JSON file

In [ ]:
from kfp.v2 import compiler
compiler.Compiler().compile(pipeline_func=pipeline,
        package_path='iris_prdictor_training_pipeline_v2.json')

#### Submit the pipeline run

In [ ]:
import google.cloud.aiplatform as aip

job = aip.PipelineJob(
    display_name="iris-predictor",
    enable_caching=False,
    template_path="iris_prdictor_training_pipeline_v2.json",
    pipeline_root=pipeline_root_path,
    parameter_values={
        'project_id': 'de-2022-366209', # makesure to use your project id 
        'data_bucket': 'data_iris_lp',  # makesure to use your data bucket name 
        'trainset_filename': 'train_set.csv',     # makesure to upload these to your data bucket from DE2022/lab4/data
        'testset_filename': 'test_set.csv',    # makesure to upload these to your data bucket from DE2022/lab4/data
        'model_repo':'model_repo_iris_lp'    # makesure to use your model bucket name 
    }
)

job.run()